In [1]:
import pixiedust
import pandas as pd
import numpy as np

from src.my_aws import S3

Pixiedust database opened successfully


In [2]:
KEY_FINAL = 'Final_Data.csv'
BUCKET = 'movie-torrents'

s3_obj = S3()
df = s3_obj.get_data(KEY_FINAL, BUCKET)

In [3]:
display(df)

Actors,Awards,BoxOffice,Country,DVD,Director,Genre,Language,Metascore,Production,Rated,Released,Runtime,Title,Type,Writer,imdbID,imdbRating,imdbVotes,Kat_Count,Pirate_Count,Extra_Count,Torrentz_Count,Torrentz_Ver_Count,Zoogle_Ver_Count,ProductionBudget,DomesticBudget,WorldGross,Total_Torrents,Runtime_Log
"Sam Worthington, Zoe Saldana, Sigourney Weaver, Stephen Lang",Won 3 Oscars. Another 85 wins & 128 nominations.,749700000.0,"UK, USA",2010-04-22,James Cameron,"Action, Adventure, Fantasy","English, Spanish",83.0,20th Century Fox,PG-13,2009-12-18,162.0,Avatar,movie,James Cameron,tt0499549,7.80000019073,944150.0,509,371,140,961,21,118.0,425000000,760507625,2783918982,2120.0,5.08759633523
"Daniel Craig, Christoph Waltz, Léa Seydoux, Ralph Fiennes",Won 1 Oscar. Another 7 wins & 31 nominations.,208777731.0,"UK, USA",2016-02-09,Sam Mendes,"Action, Adventure, Thriller","English, Spanish, Italian, German, French",60.0,Sony Pictures,PG-13,2015-11-06,148.0,Spectre,movie,"John Logan (screenplay), Neal Purvis (screenplay), Robert Wade (screenplay), Jez Butterworth (screenplay), John Logan (story by), Neal Purvis (story by), Robert Wade (story by), Ian Fleming (based on characters created by)",tt2379713,6.80000019073,313528.0,217,96,152,392,12,159.0,300000000,200074175,879620923,1028.0,4.99721227376
"Christian Bale, Gary Oldman, Tom Hardy, Joseph Gordon-Levitt",Nominated for 1 BAFTA Film Award. Another 38 wins & 101 nominations.,448130642.0,"UK, USA",2012-12-03,Christopher Nolan,"Action, Thriller","English, Arabic",78.0,Warner Bros. Pictures,PG-13,2012-07-20,164.0,The Dark Knight Rises,movie,"Jonathan Nolan (screenplay), Christopher Nolan (screenplay), Christopher Nolan (story), David S. Goyer (story), Bob Kane (characters)",tt1345836,8.39999961853,1244136.0,415,215,96,623,58,23.0,275000000,448139099,1084439099,1430.0,5.09986642782
"Johnny Depp, Armie Hammer, William Fichtner, Tom Wilkinson",Nominated for 2 Oscars. Another 4 wins & 17 nominations.,89289910.0,USA,2013-12-17,Gore Verbinski,"Action, Adventure, Western","English, North American Indian",37.0,Walt Disney Pictures,PG-13,2013-07-03,150.0,The Lone Ranger,movie,"Justin Haythe (screenplay), Ted Elliott (screenplay), Terry Rossio (screenplay), Ted Elliott (screen story), Terry Rossio (screen story), Justin Haythe (screen story)",tt1210819,6.5,193685.0,233,95,120,410,66,15.0,275000000,89302115,260002115,939.0,5.0106352941
"Taylor Kitsch, Lynn Collins, Samantha Morton, Willem Dafoe",2 wins & 8 nominations.,73058679.0,USA,2012-06-05,Andrew Stanton,"Action, Adventure, Sci-Fi",English,51.0,Walt Disney Pictures,PG-13,2012-03-09,132.0,John Carter,movie,"Andrew Stanton (screenplay), Mark Andrews (screenplay), Michael Chabon (screenplay), Edgar Rice Burroughs (based on the story ""A Princess of Mars"" by)",tt0401729,6.59999990463,222756.0,266,166,144,498,47,59.0,275000000,73058679,282778100,1180.0,4.88280192259
"Mandy Moore, Zachary Levi, Donna Murphy, Ron Perlman",Nominated for 1 Oscar. Another 9 wins & 40 nominations.,200803309.0,USA,2011-03-29,"Nathan Greno, Byron Howard","Animation, Adventure, Comedy",English,71.0,Walt Disney Pictures,PG,2010-11-24,100.0,Tangled,movie,"Dan Fogelman (screenplay), Jacob Grimm (based upon the fairy tale ""Rapunzel"" by), Wilhelm Grimm (based upon the fairy tale ""Rapunzel"" by)",tt0398286,7.80000019073,321666.0,261,170,89,361,23,32.0,260000000,200821936,586581936,936.0,4.60517018599
"Tobey Maguire, Kirsten Dunst, James Franco, Thomas Haden Church",Nominated for 1 BAFTA Film Award. Another 3 wins & 32 nominations.,336530303.0,USA,2007-10-30,Sam Raimi,"Action, Adventure","English, French",59.0,Sony Pictures,PG-13,2007-05-04,139.0,Spider-Man 3,movie,"Sam Raimi (screenplay), Ivan Raimi (screenplay), Alvin Sargent (screenplay), Sam Raimi (screen story), Ivan Raimi (screen story), Stan Lee (Marvel comic book), Steve Ditko (Marvel comic book)",tt0413300,6.19999980927,413031.0,94,84,43,155,1,35.0,258000000,336530303,894860230,412.0,4.93447393313
"Robert Downey Jr., C

In [4]:
_min = 50
_max = 200
res = len(df) - len(df[(df['Runtime'] >= _min) & (df['Runtime'] <= _max)])
print('{0} records dropped if dataset is limited to a runtime length between {1}-{2} minutes'.format(res, _min, _max))

# Drop records stated above (runtime length)
df = df[(df['Runtime'] >= _min) & (df['Runtime'] <= _max)]

# Log transform
df['Runtime_Log'] = np.log(df['Runtime'])

71 records dropped if dataset is limited to a runtime length between 50-200 minutes


In [5]:
display(df)

### - Remove movies where runtime is less than 50 (and greater than 200) minutes?
___

In [6]:
display(df)

In [7]:
ct = len(df[df['DomesticBudget'] <= 0])
print('''{0} records dropped due to Domestic Budget of 0 or less -
         majority of these films are low torrent count/indie style films'''.format(ct))

df = df[df['DomesticBudget'] > 0]

df['ProductionBudget_Log'] = np.log(df['ProductionBudget'])
df['DomesticBudget_Log'] = np.log(df['DomesticBudget'])
df['WorldGross_Log'] = np.log(df['WorldGross'])

218 records dropped due to Domestic Budget of 0 or less -
         majority of these films are low torrent count/indie style films


In [8]:
display(df)

### - Try taking log-transform of * Budget values
___

In [9]:
# Extract DVD release month and year
df['DVD_Month'] = pd.to_numeric(pd.to_datetime(df['DVD']).apply(lambda x: x.month), errors='coerce')
df['DVD_Year'] = pd.to_numeric(pd.to_datetime(df['DVD']).apply(lambda x: x.year), errors='coerce')

# Extract theaters release month and year
df['Release_Month'] = pd.to_numeric(pd.to_datetime(df['Released']).apply(lambda x: x.month), errors='coerce')
df['Release_Year'] = pd.to_numeric(pd.to_datetime(df['Released']).apply(lambda x: x.year), errors='coerce')

df.dropna(subset=['DVD_Month','DVD_Year','Release_Month','Release_Year'], inplace=True)

In [10]:
display(df)

### - Change bin count to 4 for seasonal (spring, summer, fall, winter) and to 12 for monthly
___

In [ ]:
display(df)

### - Change bin count to 4 for seasonal (spring, summer, fall, winter) and to 12 for monthly
---

In [12]:
def month_encode(df, col):
    # One hot encode months
    months = (1,2,3),(4,5,6),(7,8,9),(10,11,12)
    seasons = ['Spring','Summer','Winter','Fall']

    for mon, sea in list(zip(months, seasons)):
        _col = '{0}_{1}'.format(col.split('_')[0], sea)
        if _col in df.columns:
            del df[_col]
            
        df[_col] = 0
        df.loc[df[col].isin(mon), _col] = 1
    
    return df

In [13]:
df = month_encode(df, 'Release_Month')
df = month_encode(df, 'DVD_Month')

# Transform Genres to Select Groups

In [14]:
df['Rated_Trans'] = df['Rated'].copy()
df['Rated_Trans'].fillna(value='NR', inplace=True)
df.loc[df['Rated_Trans'].isin(['NOT RATED','UNRATED']), 'Rated_Trans'] = 'NR'
df.loc[df['Rated_Trans'].isin(['GP','TV-PG','M','M/PG','APPROVED','PASSED']), 'Rated_Trans'] = 'PG'
df.loc[df['Rated_Trans'].isin(['TV-G']), 'Rated_Trans'] = 'G'
df.loc[df['Rated_Trans'].isin(['TV-14','NC-17']), 'Rated_Trans'] = 'PG-13'
df.loc[df['Rated_Trans'].isin(['X']), 'Rated_Trans'] = 'R'
df['Rated_Trans'].unique().tolist()

['PG-13', 'PG', 'G', 'R', 'NR']

In [15]:
# Encode ratings
for rate in df['Rated_Trans'].unique():
    _col = 'Rated_{0}'.format(rate.replace('-',''))
    if _col in df.columns:
        del df[_col]
    df[_col] = 0
    df.loc[df['Rated_Trans'] == rate, _col] = 1

In [16]:
display(df)

# Group by Size of Production Company

In [17]:
df['Production_Trans'] = df['Production']
df['Production_Trans'].fillna('', inplace=True)
df['Production_Trans'] = df['Production'].str.lower().apply(lambda x: str(str(x).split('/')[0]))

In [18]:
df.loc[df['Production_Trans'].str.contains('century fox'), 'Production_Trans'] = '20th century fox'
df.loc[df['Production_Trans'].str.contains('20th century'), 'Production_Trans'] = '20th century fox'
df.loc[df['Production_Trans'].isin(['fox']), 'Production_Trans'] = '20th century fox'
df.loc[df['Production_Trans'].isin(['foxfaith']), 'Production_Trans'] = 'fox faith films'
df.loc[df['Production_Trans'].str.contains('lions'), 'Production_Trans'] = 'lionsgate'
df.loc[df['Production_Trans'].str.contains('dream'), 'Production_Trans'] = 'dreamworks'
df.loc[df['Production_Trans'].str.contains('destination'), 'Production_Trans'] = 'destination films'
df.loc[df['Production_Trans'].str.contains('yari'), 'Production_Trans'] = 'yari film group'
df.loc[df['Production_Trans'].str.contains('yash'), 'Production_Trans'] = 'yash raj films'

names = ['warner','mca','universal','paramount','sony','columbia','new line','mgm','miramax',
         'disney','buena vista','united artists','weinstein','touchstone','fox searchlight','summit',
         'relativity','magnolia','tristar','anchor bay','gramercy','open road','ifc','hollywood', 'district']

for name in names:
    df.loc[df['Production_Trans'].str.contains(name), 'Production_Trans'] = name

In [19]:
prod = df.groupby('Production_Trans').count().sort_values('Actors',ascending=False)['Actors'].reset_index()
prod.columns = [['Production_Trans', 'Prod_Count']]
# xtreme > 400
# large <= 400 x > 300
# medium <= 300 x > 100
# small <= 100 x > 20
# tiny <= 20

a = 400
b = 300
c = 100
d = 20

prod['Prod_Size'] = 'Tiny'
prod.loc[prod['Prod_Count'] > a, 'Prod_Size'] = 'Xtreme'
prod.loc[(prod['Prod_Count'] <= a) & (prod['Prod_Count'] > b), 'Prod_Size'] = 'Large'
prod.loc[(prod['Prod_Count'] <= b) & (prod['Prod_Count'] > c), 'Prod_Size'] = 'Medium'
prod.loc[(prod['Prod_Count'] <= c) & (prod['Prod_Count'] > d), 'Prod_Size'] = 'Small'

In [20]:
# Bring results into main dataframe
df = pd.merge(df, prod)

In [21]:
# One hot encode production size
for _size in df['Prod_Size'].unique().tolist():
    _col = 'Prod_Size_{0}'.format(_size)
    df[_col] = 0
    df.loc[df['Prod_Size'] == _size, _col] = 1

# Make Genres into individual features with cutoff 

In [22]:
# split strings into a numpy array
def split_to_array(ser):
    split_array = np.array(ser.strip().split(','))
    return pd.Series(split_array).apply(lambda x: x.strip())

In [23]:
def expand_as_matrix(df, col, limit):
    df[col].fillna('', inplace=True)
    df[col] = df[col].apply(lambda x: str(x))
    
    # turn numpy array into count of occurances
    ser = df[col].apply(split_to_array)
    ser = pd.Series(ser.values.ravel()).dropna()
    ser = ser.value_counts().sort_values(ascending=False)

    # convert series to dataframe
    ser = ser.reset_index()
    ser.columns = [col, 'Count']

    # Flag top features that film is listed under having
    top_features = ser[col][:limit].tolist()
    other_features = ser[col][limit:].tolist()

    for feat in top_features:
        i = '{0}_{1}'.format(col, feat)
        if i in df.columns:
            del df[i]
            
        df[feat] = 0
        df.loc[df[col].str.contains(feat), feat] = 1

    # For remaining features - flage under `Other`
    col_other = '{0}_Other'.format(col)
    
    if col_other in df.columns:
        del df[col_other]
        
    df[col_other] = 0
    for o_feat in other_features:

        df.loc[df[col].str.contains(o_feat), col_other] = 1

    # Rename columns to better explain purpose 
    rename_cols = {x: '{0}_{1}'.format(col, x) for x in top_features}

    df = df.rename(columns = rename_cols)
    return df

In [24]:
df = expand_as_matrix(df, 'Genre', 12)
df = expand_as_matrix(df, 'Country', 5)
df = expand_as_matrix(df, 'Language', 5)

/Users/bryant/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:35: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [25]:
df = df[[
 'imdbID',
 'Country_Canada',
 'Country_France',
 'Country_Germany',
 'Country_Other',
 'Country_UK',
 'Country_USA',
 'DVD_Fall',
 'DVD_Month',
 'DVD_Spring',
 'DVD_Summer',
 'DVD_Winter',
 'DVD_Year',
 'DomesticBudget',
 'DomesticBudget_Log',
 'Extra_Count',
 'Genre_Action',
 'Genre_Adventure',
 'Genre_Comedy',
 'Genre_Crime',
 'Genre_Drama',
 'Genre_Family',
 'Genre_Fantasy',
 'Genre_Horror',
 'Genre_Mystery',
 'Genre_Other',
 'Genre_Romance',
 'Genre_Sci-Fi',
 'Genre_Thriller',
 'Kat_Count',
 'Language_English',
 'Language_French',
 'Language_German',
 'Language_Italian',
 'Language_Other',
 'Language_Spanish',
 'Pirate_Count',
 'Prod_Size_Large',
 'Prod_Size_Medium',
 'Prod_Size_Small',
 'Prod_Size_Tiny',
 'Prod_Size_Xtreme',
 'ProductionBudget',
 'ProductionBudget_Log',
 'Rated_G',
 'Rated_NR',
 'Rated_PG',
 'Rated_PG13',
 'Rated_R',
 'Release_Fall',
 'Release_Month',
 'Release_Spring',
 'Release_Summer',
 'Release_Winter',
 'Release_Year',
 'Runtime',
 'Runtime_Log',
 'Torrentz_Count',
 'Torrentz_Ver_Count',
 'Total_Torrents',
 'WorldGross',
 'WorldGross_Log',
 'Zoogle_Ver_Count']]

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3549 entries, 0 to 3548
Data columns (total 63 columns):
imdbID                  3549 non-null object
Country_Canada          3549 non-null int64
Country_France          3549 non-null int64
Country_Germany         3549 non-null int64
Country_Other           3549 non-null int64
Country_UK              3549 non-null int64
Country_USA             3549 non-null int64
DVD_Fall                3549 non-null int64
DVD_Month               3549 non-null float64
DVD_Spring              3549 non-null int64
DVD_Summer              3549 non-null int64
DVD_Winter              3549 non-null int64
DVD_Year                3549 non-null float64
DomesticBudget          3549 non-null int64
DomesticBudget_Log      3549 non-null float64
Extra_Count             3549 non-null int64
Genre_Action            3549 non-null int64
Genre_Adventure         3549 non-null int64
Genre_Comedy            3549 non-null int64
Genre_Crime             3549 non-null int64
Genre_

In [27]:
df.to_csv('EDA.csv')